In [1]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession.builder
    .appName('iot_data_ingestion')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
25/06/09 09:49:53 WARN Utils: Your hostname, HP-Pavilion-15-ec2121nr resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/09 09:49:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntu/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0179cedd-fe76-41c9-a1d4-0f2feebb4609;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 162ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0 

In [3]:
turbine = spark.read.format('delta').load('../data/bronze/bronze_turbine')
health = spark.read.format('delta').load('../data/bronze/bronze_turbine_status')
sensor_hourly = spark.read.format('delta').load('../data/silver/silver_sensor_hourly')

In [8]:
(
    sensor_hourly
    .join(turbine, 'turbine_id', 'inner')
    .join(health, 'turbine_id', 'inner')
    .write.format('delta')
    .mode('overwrite')
    .option('overwriteSchema', 'true')
    .save('../data/silver/spark_turbine_training_dataset')
)

In [9]:
spark_training_dataset = spark.read.format('delta').load('../data/silver/spark_turbine_training_dataset')
spark_training_dataset.createOrReplaceTempView('spark_turbine_training_dataset')

In [11]:
spark.sql("SELECT * FROM spark_turbine_training_dataset").show(10)

+--------------------+-------------------+-------------------+------------------+--------------------+------------------+--------------------+------------------+--------------------+------------------+--------------------+------------------+--------------------+------------------+--------------------+-------+--------+-----------------+----------+--------+-------------------+---------------+----------+--------------------+----------+
|          turbine_id|   hourly_timestamp|         avg_energy|      std_sensor_E|percentiles_sensor_E|      std_sensor_C|percentiles_sensor_C|      std_sensor_B|percentiles_sensor_B|      std_sensor_A|percentiles_sensor_A|      std_sensor_F|percentiles_sensor_F|      std_sensor_D|percentiles_sensor_D|country|     lat|         location|      long|   model|              state|abnormal_sensor|  end_time|  maintenance_report|start_time|
+--------------------+-------------------+-------------------+------------------+--------------------+------------------+-----